In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from polygon import RESTClient
import numpy as np


In [2]:
EURPLN = pd.read_csv("EURPLN.csv", parse_dates=['datetime'])
GBPCHF = pd.read_csv("GBPCHF.csv", parse_dates=['datetime'])
HKDJPY = pd.read_csv("HKDJPY.csv", parse_dates=['datetime'])
NZDUSD = pd.read_csv("NZDUSD.csv", parse_dates=['datetime'])
SGDJPY = pd.read_csv("SGDJPY.csv", parse_dates=['datetime'])
USDBRL = pd.read_csv("USDBRL.csv", parse_dates=['datetime'])
USDDKK = pd.read_csv("USDDKK.csv", parse_dates=['datetime'])
USDHKD = pd.read_csv("USDHKD.csv", parse_dates=['datetime'])
USDMXN = pd.read_csv("USDMXN.csv", parse_dates=['datetime'])
USDTRY = pd.read_csv("USDTRY.csv", parse_dates=['datetime'])


C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\2343674708.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  EURPLN = pd.read_csv("EURPLN.csv", parse_dates=['datetime'])
C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\2343674708.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  NZDUSD = pd.read_csv("NZDUSD.csv", parse_dates=['datetime'])
C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\2343674708.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  USDDKK = pd.read_csv("USDDKK.csv", parse_dates=['datetime'])


In [3]:
def resample_data(df, start_date, end_date):
    # Filter data by date range
    data = df[(df['datetime'] >= start_date) & (df['datetime'] < end_date)]

    data['price_times_transactions'] = data['vw'] * data['n']

    # Group the data into the specified frequency intervals
    grouped_data = data.groupby(pd.Grouper(key='datetime', freq='6H'))

    # Calculate VWAP and Liquidity for each group
    grouped_vwap = grouped_data['price_times_transactions'].sum() / grouped_data['n'].sum()
    grouped_liquidity = grouped_data['n'].sum() / 6


    grouped_vwap = grouped_vwap.fillna(0)
    max_value = grouped_data['h'].max().fillna(0)
    min_value = grouped_data['l'].min().fillna(0)

    # Combine the results into a new DataFrame
    resampled_df = pd.DataFrame({
        'timestamp': grouped_vwap.index,
        'vwap': grouped_vwap.values,
        'liquidity': grouped_liquidity.values,
        'max': max_value,
        'min': min_value
    })

    resampled_df['volatility'] = (resampled_df['max'] - resampled_df['min']) / resampled_df['vwap']

    return resampled_df


In [4]:
start_date = "2022-01-01"
end_date = "2023-01-01"
EURPLN = resample_data(EURPLN, start_date, end_date)
GBPCHF = resample_data(GBPCHF, start_date, end_date)
HKDJPY = resample_data(HKDJPY, start_date, end_date)
NZDUSD = resample_data(NZDUSD, start_date, end_date)
SGDJPY = resample_data(SGDJPY, start_date, end_date)
USDBRL = resample_data(USDBRL, start_date, end_date)
USDDKK = resample_data(USDDKK, start_date, end_date)
USDHKD = resample_data(USDHKD, start_date, end_date)
USDMXN = resample_data(USDMXN, start_date, end_date)
USDTRY = resample_data(USDTRY, start_date, end_date)

C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price_times_transactions'] = data['vw'] * data['n']
C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price_times_transactions'] = data['vw'] * data['n']
C:\Users\Charles\AppData\Local\Temp\ipykernel_28408\3558701289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [5]:
def calculate_fd_values(df):
    fd_values = [0]
    for idx in range(1, len(df)):
        max_rate = df.iloc[idx - 1]['max']
        min_rate = df.iloc[idx - 1]['min']
        vol = df.iloc[idx - 1]['volatility']
        mean_rate = df.iloc[idx - 1]['vwap']

        keltner_upper = [mean_rate + n * 0.025 * vol for n in range(1, 101)]
        keltner_lower = [mean_rate - n * 0.025 * vol for n in range(1, 101)]

        prices = df.iloc[idx - 1:idx + 1]['vwap'].values

        crosses = calculate_fd(prices, keltner_upper, keltner_lower)

        fd = 0
        if (max_rate - min_rate) != 0:
            fd = crosses / (max_rate - min_rate)
        elif crosses == 0:
            fd = 0
        else:
            fd = 0

        fd_values.append(fd)

    df['fd'] = fd_values
    return df


def calculate_fd(prices, keltner_upper, keltner_lower):
    crosses = 0
    for i in range(1, len(prices)):
        for upper, lower in zip(keltner_upper, keltner_lower):
            if (prices[i - 1] < upper and prices[i] > upper) or (prices[i - 1] > lower and prices[i] < lower):
                crosses += 1
    return crosses

In [6]:
EURPLN = calculate_fd_values(EURPLN)
GBPCHF = calculate_fd_values(GBPCHF)
HKDJPY = calculate_fd_values(HKDJPY)
NZDUSD = calculate_fd_values(NZDUSD)
SGDJPY = calculate_fd_values(SGDJPY)
USDBRL = calculate_fd_values(USDBRL)
USDDKK = calculate_fd_values(USDDKK)
USDHKD = calculate_fd_values(USDHKD)
USDMXN = calculate_fd_values(USDMXN)
USDTRY = calculate_fd_values(USDTRY)

In [7]:
lst = [EURPLN,GBPCHF,HKDJPY,NZDUSD,SGDJPY,USDBRL,USDDKK,USDHKD,USDMXN,USDTRY]

In [1]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create a new database or connect to an existing one
db = client["currency_database"]

# Create a collection (similar to a table in relational databases) for each currency
currency_names = ['EURPLN', 'GBPCHF', 'HKDJPY', 'NZDUSD', 'SGDJPY', 'USDBRL', 'USDDKK', 'USDHKD', 'USDMXN', 'USDTRY']
collections = {
    currency: db[currency] for currency in currency_names
}

# Store the DataFrames into their respective collections in MongoDB
for currency, data in zip(currency_names, lst):
    # Convert the DataFrame to a list of dictionaries
    data_list = data.to_dict('records')

    # Insert the data into the corresponding collection
    collections[currency].insert_many(data_list)


NameError: name 'lst' is not defined

In [8]:
from pycaret.regression import *
from sklearn.model_selection import train_test_split
from pycaret.regression import compare_models, predict_model, evaluate_model
from pycaret.regression import setup, compare_models, predict_model, evaluate_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

for item in lst:
    
    data = item.copy()
    data.dropna(inplace=True)
    train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
    reg_setup = setup(data=train_data, target='vwap')

    model_comparison_results = compare_models(sort='R2', verbose=False)

    # Get the best model based on R-squared score
    best_model = compare_models(sort='R2')

    # Get the name of the best model
    best_model_name = best_model.__class__.__name__

    # Make predictions using the best model on the test data
    predictions = predict_model(best_model, data=test_data)
    
    y_true = test_data['vwap']
    y_pred = predictions['vwap']
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)




,Description,Value
0,Session id,2207
1,Target,vwap
2,Target type,Regression
3,Original data shape,"(780, 7)"
4,Transformed data shape,"(780, 9)"
5,Transformed train set shape,"(546, 9)"
6,Transformed test set shape,"(234, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0025,0.0000,0.0040,0.9974,0.0007,0.0005,0.1590
lar,Least Angle Regression,0.0025,0.0000,0.0040,0.9974,0.0007,0.0005,0.1600
br,Bayesian Ridge,0.0024,0.0000,0.0040,0.9974,0.0007,0.0005,0.1580
et,Extra Trees Regressor,0.0028,0.0000,0.0047,0.9964,0.0008,0.0006,0.1720
gbr,Gradient Boosting Regressor,0.0033,0.0000,0.0051,0.9957,0.0009,0.0007,0.1670
rf,Random Forest Regressor,0.0034,0.0000,0.0060,0.9944,0.0010,0.0007,0.1700
lightgbm,Light Gradient Boosting Machine,0.0044,0.0001,0.0083,0.9891,0.0014,0.0009,0.1660
omp,Orthogonal Matching Pursuit,0.0060,0.0001,0.0087,0.9880,0.0015,0.0013,0.1580
ada,AdaBoost Regressor,0.0067,0.0001,0.0092,0.9873,0.0016,0.0014,0.1640
dt,Decision Tree Regressor,0.0054,0.0001,0.0095,0.9841,0.0017,0.0012,0.1590


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.0027,0.0000,0.0044,0.9971,0.0008,0.0006


,Description,Value
0,Session id,5704
1,Target,vwap
2,Target type,Regression
3,Original data shape,"(751, 7)"
4,Transformed data shape,"(751, 9)"
5,Transformed train set shape,"(525, 9)"
6,Transformed test set shape,"(226, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.0090,0.0002,0.0149,0.9987,0.0007,0.0004


,Description,Value
0,Session id,4370
1,Target,vwap
2,Target type,Regression
3,Original data shape,"(758, 7)"
4,Transformed data shape,"(758, 9)"
5,Transformed train set shape,"(530, 9)"
6,Transformed test set shape,"(228, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0142,0.0006,0.0239,0.9998,0.0014,0.0009,0.1760
ridge,Ridge Regression,0.0152,0.0009,0.0266,0.9998,0.0016,0.0009,0.1710
br,Bayesian Ridge,0.0142,0.0006,0.0239,0.9998,0.0014,0.0009,0.1720
et,Extra Trees Regressor,0.0146,0.0009,0.0263,0.9998,0.0016,0.0009,0.1850
gbr,Gradient Boosting Regressor,0.0186,0.0011,0.0310,0.9997,0.0019,0.0012,0.1820
omp,Orthogonal Matching Pursuit,0.0231,0.0015,0.0368,0.9996,0.0023,0.0015,0.1690
rf,Random Forest Regressor,0.0179,0.0016,0.0332,0.9996,0.0020,0.0011,0.1790
huber,Huber Regressor,0.0183,0.0020,0.0373,0.9995,0.0021,0.0011,0.1720
lightgbm,Light Gradient Boosting Machine,0.0277,0.0031,0.0477,0.9992,0.0029,0.0018,0.1730
dt,Decision Tree Regressor,0.0258,0.0032,0.0504,0.9992,0.0030,0.0016,0.1720


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.0120,0.0003,0.0185,0.9999,0.0011,0.0007
